In [1]:
import os
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool as ThreadPool
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from functools import partial
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By    
from selenium.webdriver.support import expected_conditions as EC

In [65]:
def element_exits(xpath):
    
    try:
        driver.find_element_by_xpath(xpath)

        return True

    except:

        return False

In [2]:
pro_list = pd.read_excel('./Pro_list.xlsx')
web_list = pro_list['Website for google scholar']
web_list = web_list.to_list()
prof_name_list = pro_list['Name'].to_list()

In [3]:
def failure_crawler(ind):
    web,prof_name = web_list[ind], prof_name_list[ind]
    paper_list = []
    citation_list = []
    year_list = []
    author_list = []
    discription_list = []
    
    
    chrome_options = Options()
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    driver.get(web)
    
    print('Crawling google scholar website of Porf:{}'.format(prof_name))
    
    research_area = driver.find_element_by_xpath('//div[@id="gsc_prf_int"]').text
    print(research_area)
    if not research_area:
        research_area = ' '
    try:
        
        time.sleep(0.5)
        #year
        for year in driver.find_elements_by_xpath('//span[@class="gsc_a_h gsc_a_hc gs_ibl"]'):
            
            year_list.append(year.text)
        
        #citation
        for citation in driver.find_elements_by_xpath('//a[@class="gsc_a_ac gs_ibl"]'):
            #print(citation.text)
            citation_list.append(citation.text)
       
        
        #paper author discription   
        for each_element in driver.find_elements_by_xpath('//a[@class="gsc_a_at"]'):
            #print(each_element.text)
            paper_list.append(each_element.text)
            
            python_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gsc_a_b"]/tr[1]/td[1]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", each_element)
            driver.execute_script("arguments[0].click();", each_element)
            
            
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="gsc_vcd_table"]')))
            try:
                author = driver.find_element_by_xpath('//div[contains(text(),"Authors")]/following::*').text
            except:
                author = prof_name
            author_list.append(author)
         
            try:
                discription = driver.find_element_by_xpath('//*[@id="gsc_vcd_descr"]/div').text
                
            except:
                discription = 'None'
            discription_list.append(discription)
            #print(discription)
              
            close_button = driver.find_element_by_xpath('//*[@id="gs_md_cita-d-x"]/span[1]')
            driver.execute_script("arguments[0].scrollIntoView();", close_button)
            driver.execute_script("arguments[0].click();", close_button)
            time.sleep(1)
        
        if len(year_list) == len(citation_list) and len(paper_list)==len(year_list):
            temp_df = pd.DataFrame(
            {
                'Name': [prof_name]*len(year_list),
                'Google_Link' :[web]*len(year_list),
                'Paper': paper_list,
                'Author': author_list,
                'Citation': citation_list,
                'Year': year_list,
                'Discription': discription_list
            }
            )
            temp_df.to_csv('./prof_info.csv',index = False, mode= 'a',header= False)
            print('Successfully Crawling!')
        else:
            print('长度错了')
            print('Crawling Failure!(Store failure info)')

    except:
            print('元素抓取报错')
            print('Crawling Failure!(Store failure info)')                                                                             
    driver.quit()                                
                                                                                
                                                                                

In [5]:
pool = ThreadPool(processes=4)
pool.map(crawler,list(range(len(web_list))))
pool.close()
pool.join()

Crawling google scholar website of Porf:Jagath Rajapakse

Crawling google scholar website of Porf:Sinno Jialin Pan
Crawling google scholar website of Porf:Rui Tan
SensingsecurityInternet of Thingscyber-physical systems
Transfer LearningArtificial IntelligenceMachine Learning
Crawling google scholar website of Porf:Chng Eng Siong
SpeechDigital Signal ProcessingPattern Recognition
Successfully Crawling!
Successfully Crawling!
Successfully Crawling!
Successfully Crawling!
Crawling google scholar website of Porf:Yew-Soon Ong
Artificial & Computational intelligenceEvolutionary ComputationMachine LearningDesign Optimization
Crawling google scholar website of Porf:Bo An
Crawling google scholar website of Porf:Qian Kemao
Artificial intelligencemulti-agent systemsgame theoryreinforcement learningoptimization
Optical metrologyimage processingparallel computingcomputer vision
Crawling google scholar website of Porf:Wooi-Boon Goh
Human Computer InteractionComputer Vision
Successfully Crawling!
Suc

Crawling Failure!(Store failure info)
Successfully Crawling!
Crawling google scholar website of Porf:Mandar Chitre
Underwater acousticsunderwater communicationsunderwater robotics
Crawling Failure!(Store failure info)
Successfully Crawling!
Crawling google scholar website of Porf:Jiashi Feng
computer visionmachine learning
Crawling google scholar website of Porf:Gan Woon Seng
Active Noise ControlMachine & Deep LearningSpatial Audio
Crawling google scholar website of Porf:MH Hong
LaserPhotonics
Crawling Failure!(Store failure info)
Crawling google scholar website of Porf:Guan Yong Liang​​​
Communication TheoryCoding Theory
Successfully Crawling!
Crawling google scholar website of Porf:Eng Fong Chor
SemiconductorDevicesMicroelectronics DevicesNanoelectronic devices
Successfully Crawling!
Successfully Crawling!
Crawling google scholar website of Porf:Xiao Gong
Semiconductor DevicesRF devices and circuitsFlexible Electronic SystemsPhotonic Devices
Crawling google scholar website of Porf:Ch

In [68]:
df_fail = pd.read_csv('failure.csv',header=None)
df_fail.columns=['Name','Web']


In [69]:
web_list_fl, prof_name_list_fl = df_fail['Web'].to_list(), df_fail['Name'].to_list()

In [70]:
new_df_fail = df_fail.copy(deep=True)

In [71]:
def failure_crawler(ind):
    web,prof_name = web_list_fl[ind], prof_name_list_fl[ind]
    paper_list = []
    citation_list = []
    year_list = []
    author_list = []
    discription_list = []
    
    
    chrome_options = Options()
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    driver.get(web)
    
    print('Crawling google scholar website of Porf:{}'.format(prof_name))
    
    research_area = driver.find_element_by_xpath('//div[@id="gsc_prf_int"]').text
    print(research_area)
    if not research_area:
        research_area = ' '
    try:
        
        time.sleep(0.5)
        #year
        for year in driver.find_elements_by_xpath('//span[@class="gsc_a_h gsc_a_hc gs_ibl"]'):
            
            year_list.append(year.text)
        
        #citation
        for citation in driver.find_elements_by_xpath('//a[@class="gsc_a_ac gs_ibl"]'):
            #print(citation.text)
            citation_list.append(citation.text)
       
        
        #paper author discription   
        for each_element in driver.find_elements_by_xpath('//a[@class="gsc_a_at"]'):
            #print(each_element.text)
            paper_list.append(each_element.text)
            
            python_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gsc_a_b"]/tr[1]/td[1]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", each_element)
            driver.execute_script("arguments[0].click();", each_element)
            
            
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="gsc_vcd_table"]')))
            try:
                author = driver.find_element_by_xpath('//div[contains(text(),"Authors")]/following::*').text
            except:
                author = prof_name
            author_list.append(author)
         
            try:
                discription = driver.find_element_by_xpath('//*[@id="gsc_vcd_descr"]/div').text
                
            except:
                discription = 'None'
            discription_list.append(discription)
            #print(discription)
              
            close_button = driver.find_element_by_xpath('//*[@id="gs_md_cita-d-x"]/span[1]')
            driver.execute_script("arguments[0].scrollIntoView();", close_button)
            driver.execute_script("arguments[0].click();", close_button)
            time.sleep(1)
        
        if len(year_list) == len(citation_list) and len(paper_list)==len(year_list):
            temp_df = pd.DataFrame(
            {
                'Name': [prof_name]*len(year_list),
                'Google_Link' :[web]*len(year_list),
                'Paper': paper_list,
                'Author': author_list,
                'Citation': citation_list,
                'Year': year_list,
                'Discription': discription_list
            }
            )
            temp_df.to_csv('./prof_info.csv',index = False, mode= 'a',header= False)
            print('Successfully Crawling!')
            new_df_fail=new_df_fail.drop(new_df_fail[new_df_fail['Name']==prof_name].index)
        else:
            print('长度错了')
            print('Crawling Failure!(Store failure info)')

    except:
            print('元素抓取报错')
            print('Crawling Failure!(Store failure info)')                                                                             
    driver.quit()                                
                                                                                
                                                                                

In [72]:
pool = ThreadPool(processes=4)
pool.map(failure_crawler,list(range(len(web_list_fl))))
pool.close()
pool.join()

Crawling google scholar website of Porf:Wooi-Boon Goh
Human Computer InteractionComputer Vision
Crawling google scholar website of Porf:Cai Wentong
Parallel and Distributed ComputingModeling and Simulation
Crawling google scholar website of Porf:Erik Cambria
Crawling google scholar website of Porf:Trevor Erik CARLSONaffective computingsentiment analysisnatural language understandingcommonsense reasoningsentic computing

Computer Architecture
A handle bar metaphor for virtual object manipulation with mid-air interaction
Transparent adaptation of single-user applications for multi-user real-time collaboration
Recent Trends in Deep Learning Based Natural Language Processing
Sniper: Exploring the level of abstraction for scalable and accurate parallel multi-core simulation
New Avenues in Opinion Mining and Sentiment Analysis
Strategies for shape matching using skeletons
Crowd modeling and simulation technologies
An evaluation of high-level mechanistic core models
Affective Computing and Se

Approximation algorithms for spanner problems and directed steiner forest
A Novel Bayesian Similarity Measure for Recommender Systems.
Tight cell probe bounds for succinct boolean matrix-vector multiplication
Semantic role labeling with neural network factors
Machine learning and statistical analysis for materials science: stability and transferability of fingerprint descriptors and chemical insights
LibRec: A Java Library for Recommender Systems
An O (nϵ) space and polynomial time algorithm for reachability in directed layered planar graphs
Codra: A novel discriminative framework for rhetorical analysis
Every locally characterized affine-invariant property is testable
Evaluating the trustworthiness of advice about seller agents in e-marketplaces: A personalized approach
Streaming algorithms for computing edit distance without exploiting suffix trees
Sleep quality prediction from wearable data using deep learning
Testing linear-invariant non-linear properties
A Novel Recommendation Mod

Game balancing with ecosystem mechanism
Mmss: Multi-modal sharable and specific feature learning for rgb-d object recognition
Mystiko—blockchain meets big data
Gamelets—Multiplayer mobile games with distributed micro-clouds
Online learning asymmetric boosted classifiers for object detection
Low cost infant monitoring and communication system
Energy efficient multi-player smartphone gaming using 3D spatial subdivisioning and pvs techniques
Crawling google scholar website of Porf:CHAN Chee Yong
database systems
Efficient filtering of XML documents with XPath expressions
Image pre-conditioning for out-of-focus projector blur
Duplex narrowcasting operations for multipresent groupware avatars on mobile devices, ijwmc: Int
Dynamic lookahead mechanism for conserving power in multi-player mobile games
Shadow elimination and occluder light suppression for multi-projector displays
A multimodal virtual anatomy e-learning tool for medical education
SuperStreamer: Enabling progressive content strea

Multimodal fusion for multimedia analysis: a survey
Crawling google scholar website of Porf:Armin ABERLE
Photovoltaic materialscellsmodules
BLOCKBENCH: A Framework for Analyzing Private Blockchains
Surface passivation of crystalline silicon solar cells: a review
Crawling google scholar website of Porf:NG Hwee Tou
Natural Language ProcessingComputational Linguistics
A machine learning approach to coreference resolution of noun phrases
Blockbench: A framework for analyzing private blockchains
Shape measures for content based image retrieval: a comparison
Impact of illumination level and oxide parameters on Shockley–Read–Hall recombination at the Si‐SiO2 interface
Query and update efficient B+-tree based indexing of moving objects
Anonymous secure routing in mobile ad-hoc networks
Feature selection, perceptron learning, and a usability case study for text categorization
Overview on SiN surface passivation of crystalline silicon solar cells
Indexing the distance: An efficient method to knn

H2 optimal control
Video-based face recognition using adaptive hidden markov models
SoundUAV: Towards Delivery Drone Authentication via Acoustic Noise Fingerprinting
Broadband circularly polarized annular-ring microstrip antenna
Development of a web-based laboratory for control experiments on a coupled tank apparatus
Audio-visual integration in multimodal communication
Sensetribute smart home occupant identification via fusion across on-object sensing devices
Miniature built-in multiband antennas for mobile handsets
Hard disk drive servo systems
Image retrieval with geometry-preserving visual phrases
PAS: Prediction-Based Actuation System for City-Scale Ridesharing Vehicular Mobile Crowdsensing
Capacitively loaded circularly polarized implantable patch antenna for ISM band biomedical applications
A web-based virtual laboratory on a frequency modulation experiment
Understanding images of groups of people
SNORES: towards a less-intrusive home sleep monitoring system using wireless sensor

Wideband monopole antennas for multi-band wireless systems
Multichannel pulse-coupled-neural-network-based color image segmentation for object detection
Advances in the LED materials and architectures for energy-saving solid-state lighting toward “lighting revolution”
A learning-based approach to direction of arrival estimation in noisy and reverberant environments
Unrestrained measurement of arm motion based on a wearable wireless sensor network
A photometric investigation of ultra-efficient LEDs with high color rendering index and high luminous efficacy employing nanocrystal quantum dot luminophores
Exemplar-based sparse representation with residual compensation for voice conversion
Opportunistic routing for body area network
Morphology-tailored synthesis of tungsten trioxide (hydrate) thin films and their photocatalytic properties
The RedDots data collection for speaker recognition
A repetitive model predictive control approach for precision tracking of a linear motion system
Stimul

The coloration and degradation mechanisms of electrochromic nickel oxide
Adaptive recurrent fuzzy neural networks for active noise control
A flat lens with tunable phase gradient by using random access reconfigurable metamaterial
Ge0.83Sn0.17 p-channel metal-oxide-semiconductor field-effect transistors: Impact of sulfur passivation on gate stack quality
Germanium diffusion and nanocrystal formation in silicon oxide on silicon substrate under rapid thermal annealing
A complex virtual source approach for calculating the diffraction beam field generated by a rectangular planar source
High-efficiency sea-water monopole antenna for maritime wireless communications
A self-aligned Ni-InGaAs contact technology for InGaAs channel n-MOSFETs
Laser-scanning probe microscope based nanoprocessing of electronics materials
User-defined spectral manipulation of HRTF for improved localisation in 3D sound systems
Wideband and low-profile H-plane ridged SIW horn antenna mounted on a large conducting plane

Micromachined piezoelectric force sensors based on PZT thin films
Ordinary preserving manifold analysis for human age and head pose estimation
Low temperature CuCu thermo-compression bonding with temporary passivation of self-assembled monolayer and its bond strength enhancement
Low thermal budget Hf/Al/Ta ohmic contacts for InAlN/GaN-on-Si HEMTs with enhanced breakdown voltage
Development of piezoelectric microcantilever flow sensor with wind-driven energy harvesting capability
A framework for measuring video similarity and its application to video query by example
Successfully Crawling!
元素抓取报错
Crawling Failure!(Store failure info)
Mechanisms of ohmic contact formation and carrier transport of low temperature annealed Hf/Al/Ta on In0. 18Al0. 82N/GaN-on-Si
More than energy harvesting–Combining triboelectric nanogenerator and flexible electronics technology for enabling novel micro-/nano-systems
Uncorrelated discriminant simplex analysis for view-invariant gait signal computing
Alternat

In [76]:
temp = pd.read_csv('prof_info.csv',header=None)

In [73]:
new_df_fail

,Name,Web
0,Wooi-Boon Goh,https://scholar.google.com.sg/citations?user=w...
1,Arnab Bhattacharyya,https://scholar.google.com/citations?hl=en&use...
2,Anand BHOJAN,https://scholar.google.com/citations?hl=en&use...
3,Cai Wentong,https://scholar.google.com/citations?user=QdmI...
4,Shafiq Joty,https://scholar.google.com/citations?hl=en&use...
5,Cristina Carbunaru,https://scholar.google.com/citations?hl=en&use...
6,Erik Cambria,https://scholar.google.com/citations?user=ilSY...
7,Jie Zhang,https://scholar.google.com/citations?hl=en&use...
8,Cham Tat Jen,https://scholar.google.com/citations?user=Lx3X...
9,Trevor Erik CARLSON,https://scholar.google.com/citations?user=bUK5...


In [67]:
new_df_fail.to_csv('failure.csv',index=False,header=False)